<a href="https://colab.research.google.com/github/stemlock/w266_final_project/blob/master/NF_Base_Model_Colab_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup

In [ ]:
from google.colab import drive
from google.colab import auth
auth.authenticate_user()

drive.mount('/content/drive')
CWD = '/content/drive/My Drive/W266 Final Project/Code'

%cd $CWD

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1TVsdzndbQ4mnh6_mVPmR4xoE28WX7oHG/W266 Final Project/Code


In [ ]:
! pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-pl68o2l7
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-pl68o2l7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3231485 sha256=4ff1570860e17c16142be5cf3247ab1ab404795bb9692ad525b0d00ef1609bfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-c4mwdb5a/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfu

In [ ]:
# Imports
import os

import pandas as pd
import numpy as np
from csv import writer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import transformers
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

In [ ]:
print("Tensorflow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

Tensorflow version: 2.7.0
Transformers version: 4.13.0.dev0


In [ ]:
# Set random seed
seed = 42

## Load Data

In [ ]:
# Read in processed data (Rows with NA in the neutral_review_text had no tokens replaced)
df_train = pd.read_csv('data/model_train.csv')
df_test = pd.read_csv('data/model_test.csv')
df_train.head()

,review_id,review_score,review_text,neutral_review_text,neutral_sub_count,female_review_text,female_sub_count,male_review_text,male_sub_count,label
0,6990,1,What the hell is this? Its one of the dumbest ...,what the hell is this? its one of the dumbest ...,1,what the hell is this? its one of the dumbest ...,1,what the hell is this? its one of the dumbest ...,0,0
1,12145,1,"As you may have gathered from the title, I who...","as you may have gathered from the title, i who...",8,"as you may have gathered from the title, i who...",7,"as you may have gathered from the title, i who...",1,0
2,7457,1,"This Canadian ""movie"" is the worst ever! Stunn...","this canadian ""movie"" is the worst ever! stunn...",7,"this canadian ""movie"" is the worst ever! stunn...",5,"this canadian ""movie"" is the worst ever! stunn...",2,0
3,7324,1,Being a Film studies graduate I would like to ...,being a film studies graduate i would like to ...,2,being a film studies graduate i would like to ...,1,being a film studies graduate i would like to ...,1,0
4,7089,1,A sexually obsessed chef leads a duplicitous l...,a sexually obsessed chef leads a duplicitous l...,7,a sexually obsessed chef leads a duplicitous l...,4,a sexually obsessed chef leads a duplicitous l...,3,0


## Baseline Metrics

### Majority tokens average

In [ ]:
# Extract the male vs female majority texts
df_female_majority = df_test[df_test['male_sub_count'] > df_test['female_sub_count']]
df_male_majority = df_test[df_test['male_sub_count'] < df_test['female_sub_count']]

In [ ]:
# Average sentiment for female majority texts
print("Average female review binary sentiment:", df_female_majority['label'].mean())

Average female review binary sentiment: 0.4896039603960396


In [ ]:
# Average review score for female majority texts
print("Average female review score 1-10:", df_female_majority['review_score'].mean())

Average female review score 1-10: 5.436138613861386


In [ ]:
# Average sentiment for male majority texts
print("Average male review binary sentiment:", df_male_majority['label'].mean())

Average male review binary sentiment: 0.5035229420862691


In [ ]:
# Average review score for male majority texts
print("Average male review score 1-10:", df_male_majority['review_score'].mean())

Average male review score 1-10: 5.509709572091425


In [ ]:
# Distribution of scores across review scores for female majority texts
df_female_majority['review_score'].value_counts()/len(df_female_majority)

1     0.181683
10    0.171782
8     0.121287
4     0.121287
3     0.111386
7     0.106931
2     0.096040
9     0.089604
Name: review_score, dtype: float64

In [ ]:
# Distribution of scores across review scores for male majority texts
df_male_majority['review_score'].value_counts()/len(df_male_majority)

1     0.197457
10    0.190067
8     0.120124
4     0.105860
3     0.101564
7     0.100361
9     0.092971
2     0.091596
Name: review_score, dtype: float64

### Proportional weighted average

In [ ]:
# Extract the proportion of male vs female tokens per review
male_proportion = (df_test['female_sub_count']/df_test['neutral_sub_count'])
female_proportion = (df_test['male_sub_count']/df_test['neutral_sub_count'])

In [ ]:
# Weighted average sentiment for female tokens
(df_test['label']*female_proportion).sum()/female_proportion.sum()

0.4922782109404571

In [ ]:
# Weighted average review scores for female tokens
(df_test['review_score']*female_proportion).sum()/female_proportion.sum()

5.440864712868192

In [ ]:
# Weighted average sentiment for male tokens
(df_test['label']*male_proportion).sum()/male_proportion.sum()

0.5028605798629667

In [ ]:
# Weighted average review scores for male tokens
(df_test['review_score']*male_proportion).sum()/male_proportion.sum()

5.508159425039407

## Data Transformations

### Split data

In [ ]:
# Load data
train_texts = df_train['review_text'].values.tolist()
n_train_texts = df_train['neutral_review_text'].values.tolist()
f_train_texts = df_train['female_review_text'].values.tolist()
m_train_texts = df_train['male_review_text'].values.tolist()
train_labels = df_train['label'].values.tolist()

test_texts = df_test['review_text'].values.tolist()
n_test_texts = df_test['neutral_review_text'].values.tolist()
f_test_texts = df_test['female_review_text'].values.tolist()
m_test_texts = df_test['male_review_text'].values.tolist()
test_labels = df_test['label'].values.tolist()

In [ ]:
# Create dev set from portion of test set using split
dev_texts, test_texts, _, _ = train_test_split(test_texts, test_labels, test_size=.5, random_state=seed)
n_dev_texts, n_test_texts, _, _ = train_test_split(n_test_texts, test_labels, test_size=.5, random_state=seed)
f_dev_texts, f_test_texts, _, _ = train_test_split(f_test_texts, test_labels, test_size=.5, random_state=seed)
m_dev_texts, m_test_texts, dev_labels, test_labels = train_test_split(m_test_texts, test_labels, 
                                                                        test_size=.5, random_state=seed)

### Tokenize

In [ ]:
# Specify tokenizer and batch encode original datasets
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='tf')
dev_encodings = tokenizer(dev_texts, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='tf')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Batch encode the neutral datasets
n_train_encodings = tokenizer(n_train_texts, truncation=True, padding=True, return_tensors='tf')

In [ ]:
# Batch encode the male and female datasets
m_test_encodings = tokenizer(m_test_texts, truncation=True, padding=True, return_tensors='tf')
f_test_encodings = tokenizer(f_test_texts, truncation=True, padding=True, return_tensors='tf')

### Create TF.Datasets

In [ ]:
# Change labels list into tf.Tensors

tf_train_labels = tf.convert_to_tensor(train_labels)
tf_dev_labels = tf.convert_to_tensor(dev_labels)
tf_test_labels = tf.convert_to_tensor(test_labels)

In [ ]:
# Turn original encodings into datasets for easy batching

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    tf_train_labels
)).shuffle(10000, seed=seed).batch(16)

dev_dataset = tf.data.Dataset.from_tensor_slices((
    dict(dev_encodings),
    tf_dev_labels
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    tf_test_labels
)).batch(16)

In [ ]:
# Turn neutral encodings into datasets for easy batching
n_train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(n_train_encodings),
    tf_train_labels
)).shuffle(10000, seed=seed).batch(16)

In [ ]:
# Turn male and female encodings into datasets for easy batching
m_test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(m_test_encodings),
    tf_test_labels
)).batch(16)

f_test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(f_test_encodings),
    tf_test_labels
)).batch(16)

## Model Pipeline

### Initialize TF strategy (TPU preferred)

In [ ]:
# Initialize the TPU devices
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.113.189.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.113.189.50:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### Baseline Model Architecture

In [ ]:
# Starter function to create the model (we can improve on this when we start using more complex models)
def create_model():

  return TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

### Baseline Model v1 (5 epochs)

#### Train model

In [ ]:
# Create the model within each device scope
base_models = []
base_histories = []

num_epochs = 5
learn_rate = 1e-5

with open('./saved_results/results.csv', 'a+', newline='') as write_obj:
    csv_writer = writer(write_obj)
    csv_writer.writerow(['label', 'num_epochs', 'learn_rate', 'loss', 'accuracy', 'avg_m_pos_sent', 'avg_f_pos_sent', 'sent_diff'])

for i in range(5): 
  for label, train, dev in [('original', train_dataset, dev_dataset), ('neutral', n_train_dataset, dev_dataset)]:
    with strategy.scope():
      model = create_model()
      optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate)
      model.compile(optimizer=optimizer, 
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
      
    print(model.summary())

    history = model.fit(train, validation_data=dev, epochs=num_epochs, shuffle=False)
    base_histories.append(history)
    base_models.append(model)

    eval = model.evaluate(x=test_dataset)
    loss = eval[0]
    accuracy = eval[1]

    orig_m_logit_preds = model.predict(x=m_test_dataset).logits
    orig_f_logit_preds = model.predict(x=f_test_dataset).logits   

    m_pred_probs = tf.math.softmax(orig_m_logit_preds, axis=-1)
    f_pred_probs = tf.math.softmax(orig_f_logit_preds, axis=-1)

    model_stats = [label, num_epochs, learn_rate, loss, accuracy, np.mean(m_pred_probs[:,1]), np.mean(f_pred_probs[:,1]), (np.mean(m_pred_probs[:,1])-np.mean(f_pred_probs[:,1]))]

    with open('./saved_results/results.csv', 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(model_stats)

  print('~~~~~~Finished iteration ' + str(i) + '~~~~~~~')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_499']
You should probably TRAIN this model on a down-stream task to be able to use 

Model: "tf_distil_bert_for_sequence_classification_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_499 (Dropout)       multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]


2092/2092 [==============================] - ETA: 0s - loss: 0.1919 - accuracy: 0.9246

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]


...

2092/2092 [==============================] - 203s 73ms/step - loss: 0.2064 - accuracy: 0.9187 - val_loss: 0.2558 - val_accuracy: 0.8991
Epoch 2/5
2092/2092 [==============================] - 145s 69ms/step - loss: 0.1311 - accuracy: 0.9514 - val_loss: 0.3460 - val_accuracy: 0.8776
Epoch 3/5
2092/2092 [==============================] - 145s 69ms/step - loss: 0.0940 - accuracy: 0.9661 - val_loss: 0.2037 - val_accuracy: 0.9254
Epoch 4/5
2092/2092 [==============================] - 145s 69ms/step - loss: 0.0580 - accuracy: 0.9812 - val_loss: 0.2872 - val_accuracy: 0.9070
Epoch 5/5
262/262 [==============================] - 9s 28ms/step - loss: 0.2677 - accuracy: 0.9216


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some

Model: "tf_distil_bert_for_sequence_classification_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_679 (Dropout)       multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]


ResourceExhaustedError: ignored